In [9]:
!pip install ucimlrepo


In [11]:
import pandas as pd

data = {
    "label": ["ham", "spam", "ham", "spam", "ham", "spam", "ham", "spam", "ham", "spam"],
    "text": [
        "Hey, are we still meeting at 7pm tonight?",
        "WINNER! You have won a free ticket to Bahamas. Reply WIN to claim.",
        "Can you send me the report by tomorrow?",
        "Congratulations! You've been selected for a $1000 gift card. Click now!",
        "Thanks for the update. Talk to you later.",
        "This is not a scam! Your phone number won the prize. Call now!",
        "Lunch at my place today?",
        "Claim your free voucher now! Limited time only.",
        "Got it, I’ll finish it by today evening.",
        "Exclusive offer! Buy one get one free. Hurry!"
    ]
}

df = pd.DataFrame(data)
df.to_csv("spam_ham_synthetic_dataset.csv", index=False)
print("Dataset saved as 'spam_ham_synthetic_dataset.csv'")


Dataset saved as 'spam_ham_synthetic_dataset.csv'


In [12]:
import pandas as pd
import re, random, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

# Threshold for “spam” decision
SPAM_THRESHOLD = 0.40

# Word and character features
WORDS = [
    "make","address","all","3d","our","over","remove","internet","order","mail",
    "receive","will","people","report","addresses","free","business","email","you",
    "credit","your","font","000","money","hp","hpl","george","650","lab","labs",
    "telnet","857","data","415","85","technology","1999","parts","pm","direct","cs",
    "meeting","original","project","re","edu","table","conference"
]
CHARS = [';', '(', '[', '!', '$', '#']

def text_to_spambase_vector(text: str) -> np.ndarray:
    tokens = re.findall(r"\\b\\w+\\b", text.lower())
    n_words, n_chars = max(len(tokens), 1), max(len(text), 1)
    word_counts = {w: 0 for w in WORDS}
    for tok in tokens:
        if tok in word_counts:
            word_counts[tok] += 1
    word_freqs  = [100 * word_counts[w] / n_words for w in WORDS]
    char_freqs  = [100 * text.count(ch) / n_chars for ch in CHARS]
    caps = [len(m.group()) for m in re.finditer(r"[A-Z]+", text)]
    avg_run = np.mean(caps) if caps else 0
    longest = max(caps) if caps else 0
    total_run = sum(caps)
    return np.array(word_freqs + char_freqs + [avg_run, longest, total_run]).reshape(1, -1)

# Load UCI Spambase dataset
spambase = fetch_ucirepo(id=94)
X_num = spambase.data.features
y_num = spambase.data.targets.iloc[:, 0]
Xn_train, Xn_test, yn_train, yn_test = train_test_split(X_num, y_num, test_size=0.2, random_state=42, stratify=y_num)

def ga_fitness(alpha: float) -> float:
    return cross_val_score(MultinomialNB(alpha=alpha), Xn_train, yn_train, cv=3, scoring="accuracy").mean()

def mutate(a, lo=0.001, hi=2.0):
    return np.clip(a + np.random.normal(0, 0.1), lo, hi)

def genetic_alpha(gens=15, pop=8):
    population = [random.uniform(0.001, 2.0) for _ in range(pop)]
    for g in range(gens):
        scores = [(a, ga_fitness(a)) for a in population]
        scores.sort(key=lambda t: t[1], reverse=True)
        population = [a for a, _ in scores[: pop // 2]]
        while len(population) < pop:
            population.append(mutate(random.choice(population)))
        print(f"GA Gen {g+1:02}: best α = {scores[0][0]:.4f}  acc = {scores[0][1]:.4f}")
    return scores[0][0]

best_alpha = genetic_alpha()
num_model = MultinomialNB(alpha=best_alpha).fit(Xn_train, yn_train)
num_pred_test = num_model.predict(Xn_test)
NUM_ACC = accuracy_score(yn_test, num_pred_test)
NUM_REP = classification_report(yn_test, num_pred_test, digits=3)
NUM_CMAT = confusion_matrix(yn_test, num_pred_test)

plt.figure(figsize=(4, 4))
sns.heatmap(NUM_CMAT, annot=True, fmt="d", cmap="Blues", xticklabels=["Ham", "Spam"], yticklabels=["Ham", "Spam"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Spambase Confusion Matrix")
plt.tight_layout()
plt.savefig("spambase_cm.png")
plt.close()

# Load synthetic SMS dataset
df_sms = pd.read_csv("spam_ham_synthetic_dataset.csv")
if "tweet" in df_sms.columns and "text" not in df_sms.columns:
    df_sms = df_sms.rename(columns={"tweet": "text"})
if df_sms["label"].dtype != int:
    df_sms["label"] = df_sms["label"].map({"ham": 0, "spam": 1})

Xt_train, Xt_test, yt_train, yt_test = train_test_split(df_sms["text"], df_sms["label"], test_size=0.2, random_state=42, stratify=df_sms["label"])
tfidf = TfidfVectorizer(stop_words="english")
Xt_train_vec = tfidf.fit_transform(Xt_train)
Xt_test_vec = tfidf.transform(Xt_test)
txt_model = MultinomialNB().fit(Xt_train_vec, yt_train)
TXT_ACC = accuracy_score(yt_test, txt_model.predict(Xt_test_vec))
TXT_REP = classification_report(yt_test, txt_model.predict(Xt_test_vec), digits=3)

def classify(message: str):
    num_vec = text_to_spambase_vector(message)
    num_spam_prob = num_model.predict_proba(num_vec)[0][1]
    num_pred = int(num_spam_prob >= SPAM_THRESHOLD)
    txt_vec = tfidf.transform([message])
    txt_spam_prob = txt_model.predict_proba(txt_vec)[0][1]
    txt_pred = int(txt_spam_prob >= SPAM_THRESHOLD)
    final_label = "Spam" if (num_pred or txt_pred) else "Ham"

    metrics_md = (
        f"### Spambase numeric model\n"
        f"* α = **{best_alpha:.4f}**\n"
        f"* Accuracy = **{NUM_ACC:.4f}**\n"
        f"* P(spam) for this message = **{num_spam_prob:.3f}**\n\n"
        f"### SMS TF-IDF model\n"
        f"* Accuracy = **{TXT_ACC:.4f}**\n"
        f"* P(spam) for this message = **{txt_spam_prob:.3f}**\n\n"
        f"*Decision threshold:* **{SPAM_THRESHOLD:.2f}**\n\n"
        f"---\n"
        f"#### Spambase classification report\n"
        f"```text\n{NUM_REP}\n```"
    )
    return final_label, metrics_md, "spambase_cm.png"

demo = gr.Interface(
    fn=classify,
    inputs=gr.Textbox(lines=5, label="Enter e-mail / SMS text"),
    outputs=[
        gr.Textbox(label="Final Prediction (Spam or Ham)"),
        gr.Markdown(label="Model Metrics & Probabilities"),
        gr.Image(label="Spambase Confusion Matrix")
    ],
    title="Hybrid Spam Detector"
)

if __name__ == "__main__":
    demo.launch()

GA Gen 01: best α = 0.2895  acc = 0.7891
GA Gen 02: best α = 0.2895  acc = 0.7891
GA Gen 03: best α = 0.2895  acc = 0.7891
GA Gen 04: best α = 0.2895  acc = 0.7891
GA Gen 05: best α = 0.2895  acc = 0.7891
GA Gen 06: best α = 0.2895  acc = 0.7891
GA Gen 07: best α = 0.2895  acc = 0.7891
GA Gen 08: best α = 0.2895  acc = 0.7891
GA Gen 09: best α = 0.2895  acc = 0.7891
GA Gen 10: best α = 0.2895  acc = 0.7891
GA Gen 11: best α = 0.2895  acc = 0.7891
GA Gen 12: best α = 0.2895  acc = 0.7891
GA Gen 13: best α = 0.2895  acc = 0.7891
GA Gen 14: best α = 0.2895  acc = 0.7891
GA Gen 15: best α = 0.2895  acc = 0.7891
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6f37411d69d7defeb2.gradio.live

T